# Jacobi based Legendre simplex polyspace

In [9]:
import sys
sys.path.append("/home/tomas/Python_projects/sfepy")

In [2]:
def iter_by_order(order, dim):
    """
    Iterates over all combinations of basis functions indexes
    needed to create multidimensional basis in a way that creates hierarchical basis
    :param order: desired order of multidimensional basis
    :param dim: dimension of the basis
    :yields: tuple containing indexes, use in combine_polyvals and combine_polyvals_der
    :return: None
    """

    # nth(iter(map(lambda x: x + (order - reduce(add,x),)), range(order)), dim)
    # nth(dim, iterate(map(lambda x: x + (order - reduce(add,x),)), map(tuple, range(order))))
    # nth(2, iterate(map(lambda x: x + (order - reduce(add,x),)), map(lambda x: (x,), range(order))))
    porder = order + 1
    if dim == 1:
        for i in range(porder):
            yield (i,)
        return
    elif dim == 2:
        for k in range(porder):
            for i in range(k + 1):
                yield (i, k - i)
        return
    elif dim == 3:
        for k in range(porder):
            for j in range(k + 1):
                for i in range(j + 1):
                    yield (i, j - i, k - j)
        return

In [3]:
var("a", "b")
var("r", "s")
var("x", "y")
order = 2
D = 2

In [4]:
simplexP = []
simplexPdx = []
simplexPdy = []
for m, idx in enumerate(iter_by_order(order, D)):
    pa = jacobi_P(idx[0], 0, 0, a)
    pb = jacobi_P(idx[1], 2*idx[0] + 1, 0,b)*(1 - b)**idx[0]
    print("P_{}(a, b) = {}".format(m, pa*pb))
    polrs = (pa*pb).subs(
                    b == s, 
                    a == 2 * (1 + r) / (1 - s) - 1).simplify_rational()
    print("P_{}(r, s) = {}".format(m, polrs))
    polxy = expand(polrs.subs(r == 2*x - 1, s == 2*y - 1))
    simplexP.append(polxy)
    simplexPdx = diff(simplexP[m],x)
    simplexPdy = diff(simplexP[m],y)
    print("P_{}(x, y) = {}".format(m, simplexP[m]))
    print("dP_{}/dx = {}".format(m, simplexPdx))
    print("dP_{}/dy = {}".format(m, simplexPdy))
    print

P_0 = 1
P_0 = 1
P_0 = 1
dP_0/dx = 0
dP_0/dy = 0

P_1 = 3/2*b + 1/2
P_1 = 3/2*s + 1/2
P_1 = 3*y - 1
dP_1/dx = 0
dP_1/dy = 3

P_2 = -a*(b - 1)
P_2 = 2*r + s + 1
P_2 = 4*x + 2*y - 2
dP_2/dx = 4
dP_2/dy = 2

P_3 = 5/2*b^2 + b - 1/2
P_3 = 5/2*s^2 + s - 1/2
P_3 = 10*y^2 - 8*y + 1
dP_3/dx = 0
dP_3/dy = 20*y - 8

P_4 = -1/2*a*(5*b + 3)*(b - 1)
P_4 = (5*r + 4)*s + 5/2*s^2 + 3*r + 3/2
P_4 = 20*x*y + 10*y^2 - 4*x - 12*y + 2
dP_4/dx = 20*y - 4
dP_4/dy = 20*x + 20*y - 12

P_5 = 1/2*(3*a^2 - 1)*(b - 1)^2
P_5 = 6*r^2 + 2*(3*r + 2)*s + s^2 + 6*r + 1
P_5 = 24*x^2 + 24*x*y + 4*y^2 - 24*x - 8*y + 4
dP_5/dx = 48*x + 24*y - 24
dP_5/dy = 24*x + 8*y - 8



In [7]:
simplex_dots = [lambda f, g: integrate(f*g, (x, 0, 1)),
                lambda f, g: integrate(integrate(f*g, (y, 0, 1 - x)),(x, 0, 1)),
                lambda f, g: integrate(integrate(integrate(f*g, (z, 0, y)),(y, 0, 1 - x)),(x, 0, 1))
               ]# Fubini, yayy!

In [8]:
Np = len(simplexP)
from sympy.matrices import ones
vendM = ones(Np, Np)
for i in range(Np):
    for j in range(Np):
        vendM[i, j] = simplex_dots[D - 1](simplexP[i],
                                         simplexP[j])
vendM

Matrix([
[1/2,   0,   0,   0,   0,    0],
[  0, 1/4,   0,   0,   0,    0],
[  0,   0, 1/3,   0,   0,    0],
[  0,   0,   0, 1/6,   0,    0],
[  0,   0,   0,   0, 2/9,    0],
[  0,   0,   0,   0,   0, 8/15]])